# SAVE AND LOAD THE MODEL

在这一节，我们会学习如何保持model state，保存、加载、运行model。


In [2]:
import torch
import torch.onnx as onnx
import torchvision.models as models

## 保存和加载Model Weights

Pytorch的model在一个内部状态字典中存储学习到的parameter，叫做<kbd>state_dict</kbd>,使用<kbd>torch.save</kbd>方法可以保存这些parameter。

In [3]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Lenovo/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

加载model weights，你需要创建一个相同model的实例，然后使用<kbd>load_state_dict()</kbd>方法来加载这些parameter。

In [4]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

NOTE

be sure to call <kbd>model.eval()</kbd> method before inferencing to set the dropout and batch normalization layers to evaluation mode. Failing to do this will yield inconsistent inference results.

## 保存和加载model以及model的形状

当我们加载model weight的时候，我们需要实例化model class，因为这个class定义了network的结构，我们可能想把这个network的结构和model一起保存，我们可以将<kbd>model</kbd>(不是<kbd>model.state_dict()</kbd>)传入到save函数中。

In [5]:
torch.save(model, 'model.pth')

使用如下方法加载model：

In [6]:
model = torch.load('model.pth')

NOTE

This approach uses Python pickle module when serializing the model, thus it relies on the actual class definition to be available when loading the model.

## model导出为ONNX

Pytorch提供原生的ONNX导出支持，然而考虑到Pytorch execution graph的动态性，这个导出过程必须遍历这个execution graph来产生持久的ONNX model。出于这种考虑，需要将一个合适大小的test variable传入到导出的路线上(export routine)（在我们的例子中，我们将仿照着创建一个正确大小的zero tensor)：

In [7]:
input_image = torch.zeros((1,3,224,224))
onnx.export(model, input_image, 'model.onnx')

E:\Software\Anaconda3\envs\py38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


你可以在ONNX model上做很多事情，包括在不同平台或者不同编程语言下运行，有关ONNX更多的细节可以看这个教程[ONNX tutorial](https://github.com/onnx/tutorials).
